---

# 🏗️ Arquitectura de Autenticación Empresarial – Microsoft Foundry

> **Propiedad Intelectual © 2026 – Eduardo Osorio Venegas**  
> Todos los derechos reservados.

### 📌 Descripción del Ejercicio

Implementación de consumo seguro de un modelo desplegado en Microsoft Foundry mediante autenticación con Service Principal (Microsoft Entra ID), utilizando el flujo OAuth 2.0 Client Credentials.

El ejercicio valida:

- Emisión de token JWT vía Entra ID.
- Autorización contra endpoint de Foundry.
- Consumo del modelo mediante REST API.
- Buenas prácticas de seguridad en entornos Azure.

---

# Validar service principal

In [ ]:
from azure.identity import ClientSecretCredential
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

print("TENANT:", os.getenv("AZURE_TENANT_ID"))
print("CLIENT:", os.getenv("AZURE_CLIENT_ID"))
print("SECRET:", os.getenv("AZURE_CLIENT_SECRET"))
print("FOUNDRY_PROJECT_ENDPOINT:", os.getenv("FOUNDRY_PROJECT_ENDPOINT"))
print("MODEL_DEPLOYMENT_NAME:", os.getenv("MODEL_DEPLOYMENT_NAME"))

# Validar conexion Service principal

In [ ]:
credential = ClientSecretCredential(
    tenant_id=os.getenv("AZURE_TENANT_ID"),
    client_id=os.getenv("AZURE_CLIENT_ID"),
    client_secret=os.getenv("AZURE_CLIENT_SECRET")
)

token = credential.get_token("https://management.azure.com/.default")
print("Token obtenido correctamente")

# Instalar librerias

In [ ]:
#%pip install azure-ai-projects==2.0.0b2 openai==1.109.1

In [ ]:
#%pip install --upgrade azure-core

# Validar librerias instaladas

In [ ]:
import pkg_resources

packages = [
    "azure-ai-projects",
    "openai",
    "python-dotenv",
    "azure-identity"
]

for package in packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package} instalado - versión: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package} NO está instalado")

# Validar conexion proyecto aifoundry

In [ ]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(
    endpoint=os.getenv("FOUNDRY_PROJECT_ENDPOINT"),
    credential=credential
)

print("Conexión al AI Project exitosa")

# Listar los modelos disponible en Microsoft Foundry del proyecto

In [ ]:
deployments = project_client.deployments.list()

for d in deployments:
    print(d.name)

# Utilizar el modelo

In [ ]:
openai_client = project_client.get_openai_client()

response = openai_client.responses.create(
    model=os.getenv("MODEL_DEPLOYMENT_NAME"),
    input="que sabes de Chile"
)

print(response.output_text)